In [1]:
import numpy as np
import tensorflow as tf
import wandb

from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os
from keras.models import load_model
from sklearn.model_selection import train_test_split
from tools.config import PROCESSED_DATA_DIR, INTERIM_DATA_DIR

2025-01-20 00:29:23.804738: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 00:29:24.490464: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-01-20 00:29:24.490539: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2025-01-20 00:29:24.490547: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import keras
tf.__version__, keras.__version__

('2.11.0', '2.11.0')

In [ ]:
!nvidia-smi

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2025-01-20 00:20:30.609450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 00:20:30.684350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:20:30.712491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:20:30.712540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

True

In [3]:
tf.config.list_physical_devices('GPU')

2025-01-20 00:29:28.451084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:29:28.469687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:29:28.469743: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [4]:
# Dataset parameters
"""
dataset_path = 'smallerSampleDataset/'
reshape_size = [224,224,3]
phishing_test_size = 0.4
num_targets = 155
"""

dataset_config = {
    "dataset_path": PROCESSED_DATA_DIR / 'smallerSampleDataset/',
    "reshape_size": [224, 224, 3],
    "phishing_test_size": 0.4,
    "num_targets": 5
}

# Model parameters
"""
input_shape = [224,224,3]
margin = 2.2
new_conv_params = [5,5,512]
"""

model_config = {
    'input_shape': [224, 224, 3],
    'margin': 2.2,
    'new_conv_params': [5, 5, 512]
}

# Training parameters
""" original params
start_lr = 0.00002
output_dir = '../../../notebooks/'
saved_model_name = 'model'
save_interval = 2000
batch_size = 32
n_iter = 21000
lr_interval = 100
"""

training_config = {
    "start_lr": 0.00002,
    'output_dir': INTERIM_DATA_DIR / 'smallerSampleDataset',
    "saved_model_name": 'model',
    'save_interval': 200,
    "batch_size": 16,  
    "n_iter": 210,
    "lr_interval":100
}

In [5]:
# Dataset parameters
dataset_path = dataset_config['dataset_path']
reshape_size = dataset_config['reshape_size']
phishing_test_size = dataset_config['phishing_test_size']
num_targets = dataset_config['num_targets']

# Model parameters
input_shape = model_config['input_shape']
margin = model_config['margin']
new_conv_params = model_config['new_conv_params']

# Training parameters
start_lr = training_config['start_lr']
output_dir = training_config['output_dir']
saved_model_name = training_config['saved_model_name']
save_interval = training_config['save_interval']
batch_size =training_config['batch_size']
n_iter = training_config['n_iter']
lr_interval = training_config['lr_interval']

# Load dataset:
    - Load training screenshots per website
    - Load Phishing screenshots per website

In [5]:

def read_imgs_per_website(data_path,targets,imgs_num,reshape_size,start_target_count):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])

    all_file_names = []
    targets_list = targets.splitlines()
    count = 0
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        print(target_path)
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            try:
                img = imread(target_path+'/'+file_names[j])
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                all_labels[count,:] = i + start_target_count
                all_file_names.append(file_names[j])
                count = count + 1
            except:
                #some images were saved with a wrong extensions
                try:
                    img = imread(target_path+'/'+file_names[j],format='jpeg')
                    img = img[:,:,0:3]
                    all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                    all_labels[count,:] = i + start_target_count
                    all_file_names.append(file_names[j])
                    count = count + 1
                except:
                    print('failed at:')
                    print('***')
                    print(file_names[j])
                    break
    return all_imgs,all_labels,all_file_names

In [ ]:
# Read images legit (train)
data_path = dataset_path + 'trusted_list/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
# imgs_num = 9363
imgs_num = 420
all_imgs_train,all_labels_train,all_file_names_train = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

# Read images phishing
data_path = dataset_path + 'phishing/'
targets_file = open(data_path+'targets.txt', "r")
targets = targets_file.read()
# imgs_num = 1195
imgs_num = 160
all_imgs_test,all_labels_test,all_file_names_test = read_imgs_per_website(data_path,targets,imgs_num,reshape_size,0)

X_train_legit = all_imgs_train
y_train_legit = all_labels_train

# Split phishing to training and test, load train and test indices.
idx_test = np.load(output_dir+'test_idx.npy')
idx_train = np.load(output_dir+'train_idx.npy')
X_test_phish = all_imgs_test[idx_test,:]
y_test_phish = all_labels_test[idx_test,:]
X_train_phish = all_imgs_test[idx_train,:]
y_train_phish = all_labels_test[idx_train,:]

#otherwise, make a new split here.

# idx = np.arange(all_imgs_test.shape[0])
# X_test_phish, X_train_phish, y_test_phish, y_train_phish,idx_test,idx_train = train_test_split(all_imgs_test, all_labels_test,idx, test_size=phishing_test_size)
# np.save(output_dir+'test_idx',idx_test)
# np.save(output_dir+'train_idx',idx_train)

In [ ]:
import joblib

joblib.dump(all_imgs_train, output_dir+'all_imgs_train')
joblib.dump(all_labels_train, output_dir+'all_labels_train')

joblib.dump(all_imgs_test, output_dir+'all_imgs_test')
joblib.dump(all_labels_test, output_dir+'all_labels_test')

In [6]:
import joblib
# # Split phishing to training and test, load train and test indices.
# all_imgs_train = joblib.load(output_dir/'all_imgs_train')
# all_labels_train = joblib.load(output_dir/+'all_labels_train')

# all_imgs_test = joblib.load(output_dir/'all_imgs_test')
# all_labels_test = joblib.load(output_dir/'all_labels_test')

idx_test = np.load(output_dir/'test_idx.npy')
idx_train = np.load(output_dir/'train_idx.npy')

imgs_train_path = output_dir / 'all_imgs_train.npy'
labels_train_path = output_dir / 'all_labels_train.npy'
file_names_train_path = output_dir / 'all_file_names_train.npy'

imgs_test_path = output_dir / 'all_imgs_test.npy'
labels_test_path = output_dir / 'all_labels_test.npy'
file_names_test_path = output_dir / 'all_file_names_test.npy'

all_imgs_train = np.load(imgs_train_path)
all_labels_train = np.load(labels_train_path)
all_file_names_train = np.load(file_names_train_path)

all_imgs_test = np.load(imgs_test_path)
all_labels_test = np.load(labels_test_path)
all_file_names_test = np.load(file_names_test_path)

X_test_phish = all_imgs_test[idx_test,:]
y_test_phish = all_labels_test[idx_test,:]
X_train_phish = all_imgs_test[idx_train,:]
y_train_phish = all_labels_test[idx_train,:]

data_path = dataset_path / 'phishing'
X_train_legit = all_imgs_train
y_train_legit = all_labels_train


# Create Model

In [7]:
def define_triplet_network(input_shape, new_conv_params):

    # Input_shape: shape of input images
    # new_conv_params: dimension of the new convolution layer [spatial1,spatial2,channels]

    # Define the tensors for the three input images
    anchor_input = Input(input_shape)
    positive_input = Input(input_shape)
    negative_input = Input(input_shape)

    # Use VGG as a base model
    base_model = VGG16(weights='imagenet',  input_shape=input_shape, include_top=False)

    x = base_model.output
    x = Conv2D(new_conv_params[2],(new_conv_params[0],new_conv_params[1]),activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(2e-4)) (x)
    x = GlobalMaxPooling2D() (x)
    model = Model(inputs=base_model.input, outputs=x)

    # Generate the encodings (feature vectors) for the two images
    encoded_a = model(anchor_input)
    encoded_p = model(positive_input)
    encoded_n = model(negative_input)

    mean_layer = Lambda(lambda x: K.mean(x,axis=1))

    square_diff_layer = Lambda(lambda tensors:K.square(tensors[0] - tensors[1]))
    square_diff_pos = square_diff_layer([encoded_a,encoded_p])
    square_diff_neg = square_diff_layer([encoded_a,encoded_n])

    square_diff_pos_l2 = mean_layer(square_diff_pos)
    square_diff_neg_l2 = mean_layer(square_diff_neg)

    # Add a diff layer
    diff = Subtract()([square_diff_pos_l2, square_diff_neg_l2])
    diff = Reshape((1,)) (diff)

    # Connect the inputs with the outputs
    triplet_net = Model(inputs=[anchor_input,positive_input,negative_input],outputs=diff)

    # return the model
    return triplet_net

In [8]:
# Define the triplet loss
# Create and compile model

def custom_loss(margin):
    def loss(y_true,y_pred):
        loss_value = K.maximum(y_true, margin + y_pred)
        loss_value = K.mean(loss_value,axis=0)
        return loss_value
    return loss
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value


model = define_triplet_network(input_shape, new_conv_params)
model.summary()

from keras import optimizers
optimizer = optimizers.Adam(lr = start_lr)
model.compile(loss=custom_loss(margin),optimizer=optimizer)

2025-01-20 00:30:00.165263: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 00:30:00.166786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:30:00.166891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-01-20 00:30:00.166923: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                          

/home/jarcin/inz/src/models/visualphishnet/.venv/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
for layer in model.layers:
    print(f"Layer: {layer.name}, Trainable: {layer.trainable}")


Layer: input_1, Trainable: True
Layer: input_2, Trainable: True
Layer: input_3, Trainable: True
Layer: model, Trainable: True
Layer: lambda_1, Trainable: True
Layer: lambda, Trainable: True
Layer: subtract, Trainable: True
Layer: reshape, Trainable: True


# Triplet Sampling

In [10]:
# Order random phishing arrays per website (from 0 to 155 target)

def order_random_array(orig_arr,y_orig_arr,targets):
    sorted_arr = np.zeros(orig_arr.shape)
    y_sorted_arr = np.zeros(y_orig_arr.shape)
    count = 0
    for i in range(0,targets):
        for j in range(0,orig_arr.shape[0]):
            if y_orig_arr[j] == i:
                sorted_arr[count,:,:,:] = orig_arr[j,:,:,:]
                y_sorted_arr[count,:] = i
                count = count + 1
    return sorted_arr,y_sorted_arr

X_test_phish,y_test_phish = order_random_array(X_test_phish,y_test_phish,num_targets)
X_train_phish,y_train_phish = order_random_array(X_train_phish,y_train_phish,num_targets)


# Store the start and end of each target in the phishing set (used later in triplet sampling)
# Not all targets might be in the phishing set
def targets_start_end(num_target,labels):
    prev_target = labels[0]
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = labels[0]
    if not labels[0] == 0:
        start_end_each_target[0,0] = -1
        start_end_each_target[0,1] = -1
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[int(labels[i-1]),1] = int(i-1)
            start_end_each_target[int(labels[i]),0] = int(i)
            prev_target = labels[i]
    start_end_each_target[int(labels[-1]),1] = int(labels.shape[0]-1)

    for i in range(1,num_target):
        if start_end_each_target[i,0] == 0:
            start_end_each_target[i,0] = -1
            start_end_each_target[i,1] = -1
    return start_end_each_target

labels_start_end_train_phish = targets_start_end(num_targets,y_train_phish)
labels_start_end_test_phish = targets_start_end(num_targets,y_test_phish)


# Store the start and end of each target in the training set (used later in triplet sampling)
def all_targets_start_end(num_target,labels):
    prev_target = 0
    start_end_each_target = np.zeros((num_target,2))
    start_end_each_target[0,0] = 0
    count_target = 0
    for i in range(1,labels.shape[0]):
        if not labels[i] == prev_target:
            start_end_each_target[count_target,1] = i-1
            count_target = count_target + 1
            start_end_each_target[count_target,0] = i
            prev_target = prev_target + 1
    start_end_each_target[num_target-1,1] = labels.shape[0]-1
    return start_end_each_target

labels_start_end_train_legit = all_targets_start_end(num_targets,y_train_legit)


In [11]:
#Sample anchor, positive and negative images
def pick_first_img_idx(labels_start_end,num_targets):
    random_target = -1
    while (random_target == -1):
        random_target = np.random.randint(low = 0,high = num_targets)
        if labels_start_end[random_target,0] == -1:
            random_target = -1
    return random_target

def pick_pos_img_idx(prob_phish,img_label):
    if np.random.uniform() > prob_phish:
        class_idx_start_end = labels_start_end_train_legit[img_label,:]
        same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
        img = X_train_legit[same_idx,:]
    else:
        if not labels_start_end_train_phish[img_label,0] == -1:
            class_idx_start_end = labels_start_end_train_phish[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_phish[same_idx,:]
        else:
            class_idx_start_end = labels_start_end_train_legit[img_label,:]
            same_idx = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
            img = X_train_legit[same_idx,:]
    return img

def pick_neg_img(anchor_idx,num_targets):
    if anchor_idx == 0:
        targets = np.arange(1,num_targets)
    elif anchor_idx == num_targets -1:
        targets = np.arange(0,num_targets-1)
    else:
        targets = np.concatenate([np.arange(0,anchor_idx),np.arange(anchor_idx+1,num_targets)])
    diff_target_idx = np.random.randint(low = 0,high = num_targets-1)
    diff_target = targets[diff_target_idx]

    class_idx_start_end = labels_start_end_train_legit[diff_target,:]
    idx_from_diff_target = np.random.randint(low = class_idx_start_end[0],high = class_idx_start_end[1]+1)
    img = X_train_legit[idx_from_diff_target,:]

    return img,diff_target

# Sample batches

In [12]:
# Don't sample negative image from the same category as the positive image (e.g. google and google drive)
# Create clusters of same-company websites (e.g. all microsoft websites)


targets_file = open(data_path/'targets.txt', "r")
all_targets = targets_file.read()
all_targets = all_targets.splitlines()

def get_idx_of_target(target_name,all_targets):
    for i in range(0,len(all_targets)):
        if all_targets[i] == target_name:
            found_idx = i
            return found_idx

#targets names of parent and sub websites
target_lists = [['microsoft','ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['apple','itunes','icloud'],['google','google_drive'],['alibaba','aliexpress']]

def get_associated_targets_idx(target_lists,all_targets):
    sub_target_lists_idx = []
    parents_ids = []
    for i in range(0,len(target_lists)):
        target_list = target_lists[i]
        parent_target = target_list[0]
        one_target_list = []
        parent_idx = get_idx_of_target(parent_target,all_targets)
        parents_ids.append(parent_idx)
        for child_target in target_list[1:]:
            child_idx = get_idx_of_target(child_target,all_targets)
            one_target_list.append(child_idx)
        sub_target_lists_idx.append(one_target_list)
    return parents_ids,sub_target_lists_idx

parents_ids,sub_target_lists_idx  = get_associated_targets_idx(target_lists,all_targets)

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_ids:
        if img_label2 in sub_target_lists_idx[parents_ids.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[0]:
        if img_label2 in sub_target_lists_idx[0] or img_label2 == parents_ids[0]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[1]:
        if img_label2 in sub_target_lists_idx[1] or img_label2 == parents_ids[1]:
            if_same = 1
    elif img_label1 in sub_target_lists_idx[2]:
        if img_label2 in sub_target_lists_idx[2] or img_label2 == parents_ids[2]:
            if_same = 1
    return if_same


In [13]:
#sample triplets
def get_batch(batch_size,num_targets):

    # initialize 3 empty arrays for the input image batch
    h = X_train_legit.shape[1]
    w = X_train_legit.shape[2]
    triple=[np.zeros((batch_size, h, w,3)) for i in range(3)]

    for i in range(0,batch_size):
        img_idx_pair1 = pick_first_img_idx(labels_start_end_train_legit,num_targets)
        triple[0][i,:,:,:] = X_train_legit[img_idx_pair1,:]
        img_label = int(y_train_legit[img_idx_pair1])

        # get image for the second: positive
        triple[1][i,:,:,:] = pick_pos_img_idx(0.15,img_label)

        # get image for the thrid: negative from legit
        # don't sample from the same cluster
        img_neg,label_neg = pick_neg_img(img_label,num_targets)
        while check_if_same_category(img_label,label_neg) == 1:
            img_neg,label_neg = pick_neg_img(img_label,num_targets)

        triple[2][i,:,:,:] = img_neg

    return triple


# Training

In [14]:
def save_keras_model(model):
    model.save(output_dir/f"{saved_model_name}.h5")
    print("Saved model to disk")

In [15]:
# run = wandb.init(
#     project="Inz",
#     notes=f"VisualPhish smallerSampleDataset",
#     config={**model_config, **dataset_config, **training_config}
# )

print("Starting training process!")
print("-------------------------------------")

targets_train = np.zeros([batch_size,1])
for i in range(1, n_iter):
    inputs=get_batch(batch_size,num_targets)
    loss_value=model.train_on_batch(inputs,targets_train)
    
    print("\n ------------- \n")
    print('Iteration: '+ str(i) +'. '+ "Loss: {0}".format(loss_value))
    # run.log({"loss": loss_value})

    if i % save_interval == 0:
        save_keras_model(model)

    if i % lr_interval == 0:
        start_lr = 0.99*start_lr
        K.set_value(model.optimizer.lr, start_lr)
        # run.log({"lr": start_lr})

Starting training process!
-------------------------------------


2025-01-20 00:30:15.838850: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8904
2025-01-20 00:30:17.210758: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-20 00:30:17.210809: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.74GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-01-20 00:30:17.210818: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m


 ------------- 

Iteration: 1. Loss: 2.402111291885376

 ------------- 

Iteration: 2. Loss: 2.394646406173706

 ------------- 

Iteration: 3. Loss: 2.375261068344116

 ------------- 

Iteration: 4. Loss: 2.372143030166626

 ------------- 

Iteration: 5. Loss: 2.3833775520324707

 ------------- 

Iteration: 6. Loss: 2.346803903579712

 ------------- 

Iteration: 7. Loss: 2.317201614379883

 ------------- 

Iteration: 8. Loss: 2.3280229568481445

 ------------- 

Iteration: 9. Loss: 2.1926705837249756

 ------------- 

Iteration: 10. Loss: 2.174473762512207

 ------------- 

Iteration: 11. Loss: 2.1394362449645996

 ------------- 

Iteration: 12. Loss: 2.011240005493164

 ------------- 

Iteration: 13. Loss: 1.6103100776672363

 ------------- 

Iteration: 14. Loss: 1.9141367673873901

 ------------- 

Iteration: 15. Loss: 0.9742411375045776

 ------------- 

Iteration: 16. Loss: 1.5117285251617432

 ------------- 

Iteration: 17. Loss: 0.5781219005584717

 ------------- 

Iteration: 18

In [ ]:
path = os.path.abspath(output_dir)
run.log_model(os.path.join(path, 'model.h5'), name="VP-Model")

# Calculate Embeddings

In [16]:
shared_model = model.layers[3]

whitelist_emb = shared_model.predict(X_train_legit,batch_size=64)
np.save(output_dir/'whitelist_emb',whitelist_emb)
np.save(output_dir/'whitelist_labels',y_train_legit)

phishing_emb = shared_model.predict(all_imgs_test,batch_size=64)
np.save(output_dir/'phishing_emb',phishing_emb)
np.save(output_dir/'phishing_labels',all_labels_test)

3/3 [==============================] - 3s 1s/step


In [ ]:
artifact=wandb.Artifact(name='VisualPhish-phase-1', type='embeddings')

artifact.add_file(os.path.join(output_dir, 'whitelist_emb.npy'), 'whitelist_emb')
artifact.add_file(os.path.join(output_dir, 'whitelist_labels.npy'), 'whitelist_labels')
artifact.add_file(os.path.join(output_dir, 'phishing_emb.npy'), 'phishing_emb')
artifact.add_file(os.path.join(output_dir, 'phishing_labels.npy'), 'phishing_labels')
run.log_artifact(artifact)

In [ ]:
run.finish()